# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [10]:
class Matrix:

    def __init__(self, n, m=None):

        if isinstance(n, list):
            data = n

            if not all(isinstance(row, list) for row in data):
                raise ValueError("Matrix must be a list of lists.")

            row_length = len(data[0])
            for row in data:
                if len(row) != row_length:
                    raise ValueError("All rows must have same number of columns.")

            self.rows = len(data)
            self.cols = row_length
            self.data = [[float(val) for val in row] for row in data]

        else:
            if m is None:
                raise ValueError("Must provide both n and m.")

            self.rows = n
            self.cols = m
            self.data = [[0.0 for _ in range(m)] for _ in range(n)]

    def __getitem__(self, key):
        if isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            self.data[key] = value

    def assign(self, other):

        if isinstance(other, Matrix):
            if self.rows != other.rows or self.cols != other.cols:
                raise ValueError("Matrix sizes must match.")
            self.data = [row[:] for row in other.data]

        elif isinstance(other, list):
            if len(other) != self.rows or len(other[0]) != self.cols:
                raise ValueError("List dimensions do not match.")
            self.data = [row[:] for row in other]

        else:
            raise ValueError("Invalid assignment.")

    def scalarmul(self, c):
        return Matrix([[self.data[i][j] * c for j in range(self.cols)]
                       for i in range(self.rows)])

    def add(self, N):
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrix sizes must match.")

        return Matrix([[self.data[i][j] + N.data[i][j]
                        for j in range(self.cols)]
                       for i in range(self.rows)])

    def sub(self, N):
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrix sizes must match.")

        return Matrix([[self.data[i][j] - N.data[i][j]
                        for j in range(self.cols)]
                       for i in range(self.rows)])

    def mat_mult(self, N):
        if self.cols != N.rows:
            raise ValueError("Incompatible sizes for multiplication.")

        result = Matrix(self.rows, N.cols)

        for i in range(self.rows):
            for j in range(N.cols):
                for k in range(self.cols):
                    result[i, j] += self.data[i][k] * N.data[k][j]

        return result

    def element_mult(self, N):
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrix sizes must match.")

        return Matrix([[self.data[i][j] * N.data[i][j]
                        for j in range(self.cols)]
                       for i in range(self.rows)])

    def equals(self, N):
        if self.rows != N.rows or self.cols != N.cols:
            return False

        for i in range(self.rows):
            for j in range(self.cols):
                if self.data[i][j] != N.data[i][j]:
                    return False
        return True

    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __mul__(self, other):
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        return self.scalarmul(other)

    def __rmul__(self, other):
        return self.scalarmul(other)

    def __eq__(self, other):
        return self.equals(other)

    def __str__(self):
        return "\n".join(str(row) for row in self.data)
                    
                               
                            
                      
            


In [12]:
def constant(n, m, c):
    return Matrix([[float(c) for _ in range(m)] for _ in range(n)])
def zeros(n, m):
    return constant(n, m, 0)
def ones(n, m):
    return constant(n, m, 1)
def eye(n):
    M = zeros(n, n)
    for i in range(n):
        M[i,i] = 1.0
    return M

In [17]:
M1 = Matrix(2, 2)
print("Zero matrix from size:")
print(M1)

M2 = Matrix([[1, 2], [3, 4]])
print("\nMatrix from list:")
print(M2)

print("\nIndexing M2[0][1]:", M2[0][1])
print("Indexing M2[1,0]:", M2[1,0])

M1.assign(M2)
print("\nAfter assignment M1 = M2:")
print(M1)

print("\nOnes matrix:")
print(ones(2, 2))

print("\nIdentity matrix:")
print(eye(2))

Zero matrix from size:
[0.0, 0.0]
[0.0, 0.0]

Matrix from list:
[1.0, 2.0]
[3.0, 4.0]

Indexing M2[0][1]: 2.0
Indexing M2[1,0]: 3.0

After assignment M1 = M2:
[1.0, 2.0]
[3.0, 4.0]

Ones matrix:
[1.0, 1.0]
[1.0, 1.0]

Identity matrix:
[1.0, 0.0]
[0.0, 1.0]


In [18]:
A = Matrix([[1,2],
            [3,4]])
B = Matrix([[5,6],
            [7,8]])
C = Matrix([[2,1],
            [0,3]])
I = eye(2)

print("(AB)C:")
print((A*B)*C)

print("\nA(BC):")
print(A*(B*C))

print("\nA(B+C):")
print(A*(B+C))

print("\nA(B+C):")
print(A*(B+C)

print("nAB + AC:")
print(A*B + A*C)

print("\nAB:") 
print(A*B)


SyntaxError: '(' was never closed (3870036914.py, line 19)